## The Disapearing Walker Lake

While the loss of the Aral Sea in Kazakhstan and Lake Urmia in Iran have received a lot of attention over the last few decades, this trend is a global phenomena.  Reciently a number of __[papers](https://earthobservatory.nasa.gov/IOTD/view.php?id=91921)__ have been published including one focusing on the __[Decline of the world's saline lakes](https://www.nature.com/articles/ngeo3052)__.  Many of these lakes have lost the majority of their volume over the last century, including Walker Lake (Nevada, USA) which has lost 90 percent of its volume over the last 100 years.

The following example is intended to replicate the typical processing required in change detection studies similar to the __[Decline of the world's saline lakes](https://www.nature.com/articles/ngeo3052)__.

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import geoviews as gv
import datashader as ds
import cartopy.crs as ccrs
import pandas as pd
import glob

from holoviews.operation.datashader import regrid, shade
from bokeh.tile_providers import STAMEN_TONER

hv.extension('bokeh', width=80)

### Downloading Landsat Imae Data

To replicate this study, we first have to obtain the data from primary sources.  The conventional way to obtain Landsat image data is to download it through USGS's 
__[EarthExpoyer](https://earthexplorer.usgs.gov/)__ or NASA's __[Giovanni](https://giovanni.gsfc.nasa.gov/giovanni/)__, but to facilitate the example two images have been downloaded from EarthExployer and cached.  

The two images used by the original study are LT05_L1TP_042033_19881022_20161001_01_T1 and 
LC08_L1TP_042033_20171022_20171107_01_T1 from 1988/10/22 and 2017/10/22 respectivly.  These images contain Landsat Surface Reflectance Level-2 Science Product images.

In [ ]:
import os
lst_images = ['LT050420331988102201T1-SC20180424140822.tar.gz','LC080420332017102201T1-SC20180424140236.tar.gz']
url = 'https://s3.amazonaws.com/datashader-data/'
for fname in lst_images:
    # only download and uncompress images if they are not already on your machine
    if not os.path.exists(fname):
        import urllib.request
        print("downloading Landsat image file '%s'"%fname)
        urllib.request.urlretrieve(url+fname, fname)
        
        from subprocess import call
        call(["tar", "-xzvf", fname]) 

In [ ]:
# arbitrarily choose a small memory limit (4GB) to stress the 
# out of core processing infrastructure
from dask.distributed import Client
client = Client(memory_limit=4e9, processes=False) 
client

In [ ]:
# LANDSAT images 1988/10/22
L5_files = sorted(glob.glob("LT05_L1TP_042033_19881022_20161001_01_T1_sr_band*.tif"))
print("number of Landsat-5 files =",len(L5_files))
#L5_files

In [ ]:
# LANDSAT images 2017/10/22
L8_files = sorted(glob.glob("LC08_L1TP_042033_20171022_20171107_01_T1_sr_band*.tif"))
print("number of Landsat-8 files =",len(L8_files))
#L8_files

In [ ]:
# read in the images
#L5_img = [xr.open_rasterio(f,chunks={'x':256,'y':256}) for f in L5_files]
#print("attrs:",L5_img.shape)
#L5_img[1].attrs

In [ ]:
# rasterio creates a 3D image (1,height,width).  We only need 2D
L5_img = xr.concat([xr.open_rasterio(f,chunks={'band':1,'x':256,'y':256})[0,:,:] for f in L5_files])
print(L5_img)
print("attrs:",L5_img.attrs)
print("shape:",L5_img.shape)
print("chunks:",L5_img.chunks)

In [ ]:
def one_band(b):
    nodata = b.attrs['nodatavals']
    xs, ys = b['x'], b['y']
    b = ds.utils.orient_array(b)
    a = (np.where(np.logical_or(np.isnan(b),b<=nodata),0,255)).astype(np.uint8)    
    col, rows = b.shape
    return hv.RGB((xs, ys[::-1], b, b, b, a), vdims=list('RGBA'))

# FIXME: need to reproject from WGS84 to UTM-11N, or transform data into WGS84
%opts RGB [width=600 height=600]
tiles = gv.WMTS(STAMEN_TONER)
tiles * shade(regrid(one_band(L5_img[1].compute())), cmap=['black', 'white']).redim(x='Easting', y='Northing')